# **PYED**: Exact diagonalization routines for finite quantum systems

Copyright (C) 2017, H. U.R. Strand

The python module `pyed` implements exact diagonalization for finite fermionic many-body quantum systems, together with calculations of several response functions in imagianary time.

The many-body system is defined using `pytriqs` second-quantized operators and the response functions are stored in `pytriqs` Green's function containters.

## Hamiltonians

As an example let us solve the Hubbard atom with Hamiltonian $H = U\hat{n}_{\uparrow} \hat{n}_{\downarrow} - \mu ( \hat{n}_{\uparrow} + \hat{n}_{\downarrow})$, where $\hat{n}_\sigma = c^\dagger_\sigma c_\sigma$.


In [1]:
from pytriqs.operators import c, c_dag
up, down = 0, 1
n_up = c_dag(up, 0) * c(up, 0)
n_down = c_dag(down, 0) * c(down, 0)

U = 1.0
mu = 0.1

H = U * n_up * n_down - mu * (n_up + n_down)

print 'H =', H

H = -0.1*C^+(0,0)C(0,0) + -0.1*C^+(1,0)C(1,0) + 1*C^+(0,0)C^+(1,0)C(1,0)C(0,0)


## Thermal equilibrium solution

To solve the thermal equilibrium of the system we can diagonalize $H$ and determine the partition function $\mathcal{Z}$ (or alternatively the free energy $\Omega = -\frac{1}{\beta} \ln \mathcal{Z}$) and the many-body density matrix $\rho$ using the egenstates $|\Gamma \rangle$ and eigenvalues $E_\Gamma$ of $H$. The partition function $\mathcal{Z}$ is given by the sum of Boltzman weights

$$
\mathcal{Z} = \sum_\Gamma e^{-\beta E_\Gamma} \, ,
$$
while the many-body density matrix is given by the ket-bra Boltzman weighted sum

$$
\rho = \frac{1}{\mathcal{Z}} \sum_\Gamma e^{-\beta E_\gamma} |\Gamma \rangle \langle \Gamma|
\, .
$$

To accomplish this we pass the Hamiltonian $H$ and a list of unique annihilation opeators used in $H$ together with the inverse temperature $\beta$ to a `pyed.TriqsExactDiagonalization` class instance:

In [2]:
beta = 2.0 # inverse temperature
fundamental_operators = [c(up,0), c(down,0)]

from pyed.TriqsExactDiagonalization import TriqsExactDiagonalization
ed = TriqsExactDiagonalization(H, fundamental_operators, beta)

print r'Z =', ed.get_partition_function()
print r'\Omega =', ed.get_free_energy()
print r'\rho ='
print ed.ed.get_density_matrix()

Z = 2.9840296413
\Omega = -0.646637307852
\rho =
[[ 0.27437085  0.          0.          0.        ]
 [ 0.          0.33511731  0.          0.        ]
 [ 0.          0.          0.33511731  0.        ]
 [ 0.          0.          0.          0.05539452]]


### Thermal expectation values

Using the many-body density matrix we can evaluate the expectation value of any operator $\mathcal{O}$ by taking the trace

$$
\langle \mathcal{O} \rangle = \textrm{Tr} [ \rho \mathcal{O} ]
$$


In [3]:
print '<n_up>   =', ed.get_expectation_value(n_up)
print '<n_down> =', ed.get_expectation_value(n_down)
print '<n_up * n_down> =', ed.get_expectation_value(n_up * n_down)

<n_up>   = 0.390511834096
<n_down> = 0.390511834096
<n_up * n_down> = 0.0553945195228


## Imaginary time single-particle Green's function
We can also calculate the dynamical fluctuations of the system by computing its response functions. The simples case is the single-particle Green's function, defined as the imaginary time ordered expectation value

$$
 G_{\sigma \sigma'}(\tau) \equiv
   - \langle \mathcal{T} \, c_{\sigma}(\tau) c_{\sigma'}^\dagger(0) \rangle
 =
 - \frac{1}{\mathcal{Z}} \text{Tr}
     \left[ e^{-\beta H} c_{\sigma}(\tau_1) c_{\sigma'}^\dagger(0) \right]
$$
where the imaginary time dependent operators are defined in the Heisenberg picture $c_{\sigma}(\tau) \equiv e^{\tau H} c_{\sigma} e^{-\tau H}$ and $c^\dagger_{\sigma}(\tau) \equiv e^{\tau H} c^\dagger_{\sigma} e^{-\tau H}$.

To calculate $G(\tau)$ we first create `pytriqs.GfImTime` instance to store the result and pass it to our ED solver instance:

In [8]:
from pytriqs.gf import GfImTime
g_tau = GfImTime(name=r'$g$', beta=beta, statistic='Fermion', n_points=50, indices=[1])    
ed.set_g2_tau(g_tau, c(up,0), c_dag(up,0))

import matplotlib.pyplot as plt
from pytriqs.plot.mpl_interface import oplot

plt.figure(); oplot(g_tau); plt.savefig('figure_g_tau.png')

![Single-particle Green's function](figure_g_tau.png)

In [7]:
from pytriqs.gf import GfImTime
densdens_tau = GfImTime(name=r'$\langle n(\tau) n(0) \rangle$', beta=beta, statistic='Boson', n_points=50, indices=[1])    
ed.set_g2_tau(densdens_tau, n_up, n_down)

plt.figure(); oplot(densdens_tau); plt.savefig('figure_densdens_tau.png')

![Density density response function](figure_densdens_tau.png)

In [10]:
from pytriqs.gf import GfImFreq
g_iwn = GfImFreq(name=r'$g$', beta=beta, statistic='Fermion', n_points=10, indices=[1])
ed.set_g2_iwn(g_iwn, c(up,0), c_dag(up,0))

plt.figure(); oplot(g_iwn); plt.savefig('figure_g_iwn.png')

![Single-particle Green's function in imaginary frequency](figure_g_iwn.png)